In [1]:
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora, models
import gensim
import pandas as pd

In [2]:
df_new = pd.read_csv('Vodaphone_review_dataset.csv')

df_new.head()

,Rating,Review,Name,Location,Date,Clean_Review,Sentiment,Sentiment Label,Churn_Keyword,Churn,Review_Date,Review_Time,Weekday,Month,Review_Length,Word_Count
0,1,"Trying to buy broadband through Uswitch, then ...",Alfie Calas,GB,2025-06-05 22:03:24+00:00,trying to buy broadband through uswitch then h...,-0.020000,neutral,1,1,2025-06-05,22:03:24,Thursday,June,280,48
1,5,Sona did a great job either my trade in and co...,Julliette,GB,2025-06-05 21:51:17+00:00,sona did a great job either my trade in and co...,0.700000,positive,0,0,2025-06-05,21:51:17,Thursday,June,68,13
2,1,"One of the worst , if not the worst when it co...",Vlad Ureche,GB,2025-06-05 21:28:36+00:00,one of the worst if not the worst when it come...,-0.200000,neutral,0,1,2025-06-05,21:28:36,Thursday,June,154,34
3,1,How those people get 4.6 rate that’s a joke my...,Adam Farbotko,GB,2025-06-05 21:14:08+00:00,how those people get rate thats a joke my full...,0.275000,positive,0,1,2025-06-05,21:14:08,Thursday,June,240,44
4,1,Held to ransom by a ‘reputable’ company. Purch...,Chloe,GB,2025-06-05 20:24:59+00:00,held to ransom by a reputable company purchase...,-0.003634,neutral,1,1,2025-06-05,20:24:59,Thursday,June,1620,320


In [3]:
complaints_df = df_new[df_new['Sentiment Label'].str.lower() != 'positive'].copy()

praise_df = df_new[df_new['Sentiment Label'].str.lower() == 'positive'].copy()


In [4]:
nltk.download('punkt_tab', download_dir='/home/codespace/nltk_data')
nltk.download('stopwords', download_dir='/home/codespace/nltk_data')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# Preprocessing text

stop_words = set(stopwords.words('english'))
def preprocess(text):
    tokens = word_tokenize(text.lower())
    return [word for word in tokens if word.isalpha() and word not in stop_words]

complaints_df['tokens'] = complaints_df['Clean_Review'].apply(
    lambda x: preprocess(x) if pd.notnull(x) else []
)


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\deela/nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\share\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\lib\\nltk_data'
    - 'C:\\Users\\deela\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
# Creating dictionary and corpus
dictionary = corpora.Dictionary(complaints_df['tokens'])
corpus = [dictionary.doc2bow(text) for text in complaints_df['tokens']]


In [ ]:
# Train LDA model (start with 5 topics)

lda_model = gensim.models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=5,
    passes=10,
    random_state=42
)


In [ ]:
# Displaying the topics found by the LDA model

topics = lda_model.print_topics(num_words=10)
for idx, topic in topics:
    print(f"Topic {idx+1}: {topic}")

Topic 1: 0.010*"router" + 0.008*"wifi" + 0.007*"vodafone" + 0.006*"fibre" + 0.005*"would" + 0.004*"still" + 0.004*"vodaphone" + 0.004*"like" + 0.004*"devices" + 0.004*"days"
Topic 2: 0.054*"helpful" + 0.016*"vodafone" + 0.014*"staff" + 0.013*"service" + 0.012*"store" + 0.012*"extremely" + 0.010*"really" + 0.009*"phone" + 0.008*"professional" + 0.008*"thank"
Topic 3: 0.038*"helpful" + 0.029*"phone" + 0.021*"new" + 0.017*"store" + 0.017*"helped" + 0.013*"thank" + 0.013*"sorted" + 0.012*"sim" + 0.011*"really" + 0.010*"everything"
Topic 4: 0.030*"vodafone" + 0.013*"service" + 0.011*"would" + 0.010*"broadband" + 0.008*"customer" + 0.006*"internet" + 0.005*"time" + 0.005*"days" + 0.005*"issue" + 0.004*"even"
Topic 5: 0.016*"vodafone" + 0.014*"phone" + 0.012*"contract" + 0.012*"service" + 0.011*"get" + 0.011*"customer" + 0.008*"told" + 0.008*"broadband" + 0.008*"call" + 0.007*"new"


In [ ]:
# getting the dominant topic for each document

def get_topic(doc):
    bow = dictionary.doc2bow(doc)
    topic_probs = lda_model.get_document_topics(bow)
    dominant_topic = max(topic_probs, key=lambda x: x[1])[0]
    return dominant_topic

In [ ]:
complaints_df['Topic'] = complaints_df['tokens'].apply(get_topic)

In [ ]:
complaints_df.head()

,Rating,Review,Name,Location,Date,Clean_Review,Sentiment,Sentiment Label,Churn_Keyword,Churn,Review_Date,Review_Time,Weekday,Month,Review_Length,Word_Count,tokens,Topic
0,1,"Trying to buy broadband through Uswitch, then ...",Alfie Calas,GB,2025-06-05 22:03:24+00:00,trying to buy broadband through uswitch then h...,-0.020000,neutral,1,1,2025-06-05,22:03:24,Thursday,June,280,48,"[trying, buy, broadband, uswitch, multiple, er...",3
2,1,"One of the worst , if not the worst when it co...",Vlad Ureche,GB,2025-06-05 21:28:36+00:00,one of the worst if not the worst when it come...,-0.200000,neutral,0,1,2025-06-05,21:28:36,Thursday,June,154,34,"[one, worst, worst, comes, signal, get, n, mes...",4
4,1,Held to ransom by a ‘reputable’ company. Purch...,Chloe,GB,2025-06-05 20:24:59+00:00,held to ransom by a reputable company purchase...,-0.003634,neutral,1,1,2025-06-05,20:24:59,Thursday,June,1620,320,"[held, ransom, reputable, company, purchased, ...",4
12,5,Mohamed was brillant and fixed my phone within...,Romy Aitken,GB,2025-06-05 18:46:06+00:00,mohamed was brillant and fixed my phone within...,0.100000,neutral,0,0,2025-06-05,18:46:06,Thursday,June,97,18,"[mohamed, brillant, fixed, phone, within, mins...",4
13,1,This is the complaint I submitted to Vodafone:...,Adriana Castorina,GB,2025-06-05 18:44:57+00:00,this is the complaint i submitted to vodafone ...,0.067885,neutral,1,1,2025-06-05,18:44:57,Thursday,June,1513,295,"[complaint, submitted, vodafone, move, flat, o...",3


In [ ]:
praise_df.head()

,Rating,Review,Name,Location,Date,Clean_Review,Sentiment,Sentiment Label,Churn_Keyword,Churn,Review_Date,Review_Time,Weekday,Month,Review_Length,Word_Count
1,5,Sona did a great job either my trade in and co...,Julliette,GB,2025-06-05 21:51:17+00:00,sona did a great job either my trade in and co...,0.700000,positive,0,0,2025-06-05,21:51:17,Thursday,June,68,13
3,1,How those people get 4.6 rate that’s a joke my...,Adam Farbotko,GB,2025-06-05 21:14:08+00:00,how those people get rate thats a joke my full...,0.275000,positive,0,1,2025-06-05,21:14:08,Thursday,June,240,44
5,5,Excellent customer service. Vinnie was very he...,Farida Ariori,GB,2025-06-05 20:05:30+00:00,excellent customer service vinnie was very hel...,0.600000,positive,0,0,2025-06-05,20:05:30,Thursday,June,62,9
6,5,Excellent service from Rushabh! He explained e...,kawater alismaeel,GB,2025-06-05 19:52:59+00:00,excellent service from rushabh he explained ev...,0.433333,positive,0,0,2025-06-05,19:52:59,Thursday,June,250,42
7,5,Steven listened then provided a number of opti...,Dylan Owen,GB,2025-06-05 19:43:30+00:00,steven listened then provided a number of opti...,0.350000,positive,0,0,2025-06-05,19:43:30,Thursday,June,175,30


In [ ]:
df_new.Review

0       Trying to buy broadband through Uswitch, then ...
1       Sona did a great job either my trade in and co...
2       One of the worst , if not the worst when it co...
3       How those people get 4.6 rate that’s a joke my...
4       Held to ransom by a ‘reputable’ company. Purch...
                              ...                        
9974    absolutely awful, been with vodaphone for 4 mo...
9975    Had a great experience- until we decided to mo...
9976    Karan and Isaac were both amazing. Give them a...
9977                          Thank you mani your amazing
9978    16 Years of Loyalty, But the Last 1.5 Years Ha...
Name: Review, Length: 9979, dtype: object

In [ ]:
# Defining responses for each topic

topic_intents = {
    0: "Router or WiFi Issue",
    1: "Staff and Service Feedback",
    2: "Phone or SIM Setup",
    3: "Broadband or Internet Problem",
    4: "Contract or Plan Concern"
}

topic_responses = {
    0: "It seems you're having trouble with your router or WiFi. Let's work together to get that sorted quickly.",
    1: "Thanks for your feedback on our staff and service. We’ll review your concerns with the relevant team.",
    2: "Appreciate you sharing your experience—let’s make sure your new phone or SIM setup is working as expected.",
    3: "We're sorry about the broadband or internet issues you've faced. We’re investigating and working to improve reliability.",
    4: "It looks like there’s frustration with your contract or service plan. We’ll help clarify and resolve that for you."
}



In [ ]:
# Mapping topics to responses

complaints_df['Response'] = complaints_df['Topic'].map(topic_responses)


In [ ]:
complaints_df.Response

0       We're sorry about the broadband or internet is...
2       It looks like there’s frustration with your co...
4       It looks like there’s frustration with your co...
12      It looks like there’s frustration with your co...
13      We're sorry about the broadband or internet is...
                              ...                        
9971    Thanks for your feedback on our staff and serv...
9973    Appreciate you sharing your experience—let’s m...
9974    It looks like there’s frustration with your co...
9975    It looks like there’s frustration with your co...
9978    It looks like there’s frustration with your co...
Name: Response, Length: 3140, dtype: object

In [ ]:
def generate_response(text, complaints_df):
    # Look up sentiment from the dataset
    sentiment_row = complaints_df[complaints_df['Clean_Review'] == text]
    
    if not sentiment_row.empty:
        sentiment = sentiment_row['Sentiment Label'].values[0].lower()
    else:
        sentiment = "neutral"  # fallback if not found

    if sentiment == "positive":
        return "Thanks for your feedback! We're glad you're happy with our service 😊"
    else:
        tokens = preprocess(text)
        bow = dictionary.doc2bow(tokens)
        topic_probs = lda_model.get_document_topics(bow)
        dominant_topic = max(topic_probs, key=lambda x: x[1])[0]
        return topic_responses.get(dominant_topic, "Thank you for reaching out.")


In [ ]:
sample_review = complaints_df['Clean_Review'].sample(1).to_list()

In [ ]:
sample_review


['really helpful in store thanks to leon']

In [ ]:
response = generate_response(sample_review[0], complaints_df)
print("Generated Response:", response)

Generated Response: Appreciate you sharing your experience—let’s make sure your new phone or SIM setup is working as expected.


In [ ]:
dictionary.save("lda_dictionary.dict")


In [ ]:
import pickle
lda_model.save("lda_model.gensim")
